# TẠO CÁC TẬP DỮ LIỆU TRAIN, TEST (SPLITS)

1. Yêu cầu chung: Tạo ra các splits, mỗi split tương ứng với một tập dữ liệu train - test
  + Bài học lý thuyết (để trả lời cho các câu hỏi như vì sao phải cần việc này, thực hiện việc này như thế nào):
    - https://www.kaggle.com/code/satishgunjal/tutorial-k-fold-cross-validation
    - https://machinelearningmastery.com/training-validation-test-split-and-cross-validation-done-right/
    - https://machinelearningmastery.com/how-to-configure-k-fold-cross-validation/

  <IMG SRC = 'https://raw.githubusercontent.com/satishgunjal/images/master/KFold_Cross_Validation.png'>
2. Yêu cầu cụ thể:
- Input:
    + Thư mục cha chứa các thư mục con - mỗi thư mục con tương ứng với tên của từng hiệu xe (Honda, Suzuki, VinFast, Yamaha, Others). Ví dụ: https://drive.google.com/drive/u/1/folders/1Uj0V9URNHpzSHeXHSB89AoGCjGki8Yra
    + Các ảnh được đặt tên theo quy ước: các tập tin ảnh theo quy ước
    + Số splits NumSplits - mặc định NumSplits=5 (tương đương 5-fold CV)
- Output:
    + File CarDataset.csv - Tập tin chứa tất cả ảnh của dataset
      - Chương trình sẽ scan qua cây thư mục để tìm tất cả các ảnh (chỉ chọn các ảnh có định dạng + phần mở rộng là .jpg)
      - Mỗi dòng sẽ có các thông tin cách nhau bằng dấu phẩy, theo quy ước: ImageFullPath, CategoryID
            - ImageFullPath ở dạng <Thư mục Hiệu xe>/<Tên ảnh>. Ví dụ: Honda/2024123.Honda.1.jpg.
            - CategoryID là số nguyên thuộc [0..5] theo quy ước
              - 0: Others
              - 1: Honda
              - 2: Suzuki
              - 3: Yamaha
              - 4: VinFast     
    + File CarDataset-Splits-[1..5]-[Train/Test].csv - Phân chia thành các splits, mỗi split gồm các ảnh được chia thành thành 2 tập Train - Test
      + Chương trình sẽ đọc dữ liệu từ file CarDataset.csv, sau đó với mỗi hiệu xe, chia ngẫu nhiên thành 5 tập dữ liệu. Lưu ý là phải chia theo hiệu xe, để đảm bảo dữ liệu Train/Test có dữ liệu của các hiệu xe.
      + Từ 5 tập dữ liệu chia ngẫu nhiên theo các hiệu xe Xij (i là thứ tự tập dữ liệu, j là CategoryID), gom lại thành 5 tập dữ liệu lớn hơn Xi, sao cho mỗi tập dữ liệu này chứa đủ dữ liệu của tất cả các hiệu xe. Tức là Xi = Union(Xij)       
      + Từ 5 tập dữ liệu Xi này (tương ứng với FOLDi ở trong hình vẽ trên), tạo ra 5 splits Split-i
      + Với mỗi bộ dữ liệu Split-i, ghi xuống thành 2 tập tin tương ứng với Train, Test. Ví dụ Split-1 thì ghi thành tập tin  CarDataset-Splits-1-Train.csv và CarDataset-Splits-1-Test.csv. Tập Train gồm 4 bộ, Test gồm bộ còn lại. Ví dụ,
          + Split-1: tập Train sẽ gồm X2, X3, X4, X5, tập Test là X1
          + Split-5: tập Train sẽ gồm X1, X2, X3, X4, tập Test là X5
      + Mỗi dòng sẽ có các thông tin cách nhau bằng dấu phẩy, theo quy ước: ImageFullPath, CategoryID
- Lưu ý:
  - Nên viết thêm các cell
    - Hiển thị danh sách các tên tập tin ảnh trong từng Split-Train/Test,
    - Thống kê các ảnh cho từng CategoryID trong mỗi Split-Train/Test
  - Cần có chú thích
3. Nộp bài: SV share notebook. Các bài nộp sớm sẽ được full điểm. Deadline: TBA
4. Bài làm đạt yêu cầu sẽ được paste vào notebook với ghi nhận đóng góp từ tác giả.

## Thông tin của tác giả, ngày cập nhật

In [1]:
import os
import pandas as pd
from sklearn.model_selection import StratifiedKFold

In [2]:
CATEGORY_MAP = {
    "Others": 0,     
    "Honda": 1,      
    "Hyundai": 2,    
    "KIA": 3,        
    "Mazda": 4,      
    "Mitsubishi": 5, 
    "Suzuki": 6,     
    "Toyota": 7,     
    "VinFast": 8,    
}

In [3]:
def create_car_dataset_csv(root_dir, output_file):
    """
    Quét qua cây thư mục để tìm tất cả các ảnh có định dạng hợp lệ.
    Lưu thông tin (ImageFullPath, CategoryID) vào file CSV.

    Parameters:
    - root_dir (str): Đường dẫn đến thư mục gốc chứa các thư mục con theo CATEGORY_MAP.
    - output_file (str): Đường dẫn và tên tệp CSV đầu ra.

    Output:
    - Tạo file CSV với hai cột: ImageFullPath (đường dẫn ảnh tương đối) và CategoryID.
    """
    valid_extensions = {".jpg", ".jpeg", ".png"} 
    rows = []  

    for folder_name, category_id in CATEGORY_MAP.items():
        folder_path = os.path.join(root_dir, folder_name)  
        if os.path.exists(folder_path):  
            for file_name in os.listdir(folder_path):  
                if any(file_name.lower().endswith(ext) for ext in valid_extensions):
                    image_path = f"{folder_name}/{file_name}"
                    rows.append([image_path, category_id])

    df = pd.DataFrame(rows, columns=["ImageFullPath", "CategoryID"])  
    df.to_csv(output_file, index=False)  

    print(f"CarDataset.csv đã được tạo với {len(df)} ảnh.")

In [4]:
from sklearn.model_selection import StratifiedKFold
import pandas as pd
import os

def create_splits(input_file, num_splits=5, output_dir="."):
    """
    Chia dữ liệu từ CarDataset.csv thành 5 tập Train/Test theo 5-fold cross-validation.
    
    Parameters:
    - input_file (str): Đường dẫn đến file CSV đầu vào chứa dữ liệu (gồm ImageFullPath và CategoryID).
    - num_splits (int): Số lượng splits (folds), mặc định là 5.
    - output_dir (str): Đường dẫn đến thư mục lưu trữ các file Train/Test, mặc định là thư mục hiện tại.
    """
    df = pd.read_csv(input_file)

    skf = StratifiedKFold(n_splits=num_splits, shuffle=True, random_state=42)  

    for fold, (train_idx, test_idx) in enumerate(skf.split(df, df["CategoryID"]), 1):
        train_df = df.iloc[train_idx]  
        test_df = df.iloc[test_idx]   

        train_file = os.path.join(output_dir, f"CarDataset-Splits-{fold}-Train.csv")
        test_file = os.path.join(output_dir, f"CarDataset-Splits-{fold}-Test.csv")

        train_df.to_csv(train_file, index=False)
        test_df.to_csv(test_file, index=False)

        print(f"Split {fold}: Train ({len(train_df)}), Test ({len(test_df)}) đã được lưu.")


In [5]:
def display_statistics(output_dir, num_splits=5):
    """
    Hiển thị thống kê số lượng ảnh theo từng CategoryID trong từng Split (Train/Test).

    Parameters:
    - output_dir (str): Đường dẫn đến thư mục chứa các tệp CSV của các fold Train/Test.
    - num_splits (int): Số lượng splits (folds), mặc định là 5.
    """
    for fold in range(1, num_splits + 1):
        train_file = os.path.join(output_dir, f"CarDataset-Splits-{fold}-Train.csv")
        test_file = os.path.join(output_dir, f"CarDataset-Splits-{fold}-Test.csv")

        train_df = pd.read_csv(train_file)
        test_df = pd.read_csv(test_file)

        print(f"--- Split {fold} ---")
        print(f"Train distribution:\n{train_df['CategoryID'].value_counts()}")
        print(f"Test distribution:\n{test_df['CategoryID'].value_counts()}")

In [6]:
if __name__ == "__main__":
    root_dir = r"D:\ML\Public"  
    output_dir = r"D:\ML\Public"  
    os.makedirs(output_dir, exist_ok=True)  

    dataset_file = os.path.join(output_dir, "CarDataset.csv")
    create_car_dataset_csv(root_dir, dataset_file)

    create_splits(dataset_file, num_splits=5, output_dir=output_dir)

    display_statistics(output_dir, num_splits=5)

CarDataset.csv đã được tạo với 7142 ảnh.
Split 1: Train (5713), Test (1429) đã được lưu.
Split 2: Train (5713), Test (1429) đã được lưu.
Split 3: Train (5714), Test (1428) đã được lưu.
Split 4: Train (5714), Test (1428) đã được lưu.
Split 5: Train (5714), Test (1428) đã được lưu.
--- Split 1 ---
Train distribution:
CategoryID
6    2008
7    1104
3     552
2     402
4     349
1     347
0     343
8     333
5     275
Name: count, dtype: int64
Test distribution:
CategoryID
6    502
7    276
3    137
2    101
4     88
1     87
0     86
8     83
5     69
Name: count, dtype: int64
--- Split 2 ---
Train distribution:
CategoryID
6    2008
7    1104
3     551
2     403
4     349
1     347
0     343
8     332
5     276
Name: count, dtype: int64
Test distribution:
CategoryID
6    502
7    276
3    138
2    100
4     88
1     87
0     86
8     84
5     68
Name: count, dtype: int64
--- Split 3 ---
Train distribution:
CategoryID
6    2008
7    1104
3     551
2     403
4     350
1     347
0     343
8 